T-test for the difference of two means

In [ ]:
from scipy import stats
import numpy as np

s0 = df.loc[(df['col']=='var_in_col') & (df['another_column']==0)]
s0 = s0['column_to_comp']
s1 = df.loc[(df['col']=='var_in_col') & (df['another_column']==1)]   
s1 = s1['column_to_comp']

print("Title:",stats.ttest_ind( s0.dropna(), s1.dropna(), equal_var = False))

In [ ]:
# correlation matrix
corr = df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
#plt.show()

There are two types of probability distributions, discrete and continuous probability distributions.
Discrete probability distributions are also called as probability mass functions. Some examples of discrete probability distributions are  Bernoulli distribution,  Binomial distribution, Poisson distribution and Geometric distribution.
Continuous probability distributions also known as probability density functions, they are functions that take on continuous values (e.g. values on the real line). Examples include the normal distribution, the exponential distribution and the beta distribution.
To understand more about discrete and continuous random variables, watch Khan academies probability distribution videos.

## CDF

https://seaborn.pydata.org/generated/seaborn.distplot.html

Example description:

Probability of red cards with respect to players within each skin rating.
Plotting CUmulative Distribution Function of the distribution of players in each skin rating with respect to the number of red cards they recieved.

Upon visual inspection of the CDF plots for both rater one and rater two, the median number of red cards is slightely higher for skin=5 compared to the other skin ratings. Although there is a slight visual difference for skin=5 compared to the rest, the difference looks quite small. Therefore I deceded to test these distributions.

In [ ]:
import seaborn as sns

sns.distplot(rater1['1'],kde_kws={'cumulative':True, 'clip':(-.5,6), "label": "skin=1"}, hist=False)
sns.distplot(rater1['2'],kde_kws={'cumulative':True, 'clip':(-.5,6), "label": "skin=2"}, hist=False)
sns.distplot(rater1['3'],kde_kws={'cumulative':True, 'clip':(-.5,6), "label": "skin=3"}, hist=False)
sns.distplot(rater1['4'],kde_kws={'cumulative':True, 'clip':(-.5,6), "label": "skin=4"}, hist=False)
sns.distplot(rater1['5'],kde_kws={'cumulative':True, 'clip':(-.5,6), "label": "skin=5"}, hist=False)


sns.plt.show()

Normality
A T-test assumes that the sample distribution is normal. The histogram plot below shows that the sample distribution is not normal. A Box-Cox power transform could be used if all values were positive. An alternative method that does not make this assumption is the Yeo-Johnson transformation. For now I will resort to using a non-parametric test.

In [ ]:
data = df.groupby(['playerShort']).sum()['redCards']
sns.distplot(data,kde_kws={'clip':(-.5,6), "label": "ALL data for rater 1"})

Mann-Whitney U Test
The Mann-Whitney U test is a non-parametric statistical significance test for determining whether two independent samples were drawn from a population with the same distribution. In This case, the null hypothesis is that there is no difference between the distributions of sampled red cards for different skin ratings. Rejection of this hypothesis suggests that there is likely some difference between the samples. We reject the null hypothesis if the p-value <= 0.05.

The results for rater 1 below suggest that:

Players with skin rating 5 are more likely to get red cards compared to skin rating 4, 3, 2, and 1.
Players of skin rating 4 are no more likely to get red cards compared to skin rating 3, 2, and 1.
Players of skin rating 3 are no more likely to get red cards compared to skin ratings 2 and 1.
Players of skin rating 2 are more likely to get red cards compared to skin rating 1.
The results for rater 2 were similar.

In [ ]:
from scipy.stats import mannwhitneyu
print "Rater 1"
for i in range(1,5):
  
  t2, p2 = mannwhitneyu(rater1[str(i)],rater1['5'])
  print "Skin " + str(i) + " vs. Skin 5 p-value: %1.2f " % p2 
  
print ""

for i in range(1,4):

  t2, p2 = mannwhitneyu(rater1[str(i)],rater1['4'])
  print "Skin " + str(i) + " vs. Skin 4 p-value: %1.2f " % p2


print ""  

for i in range(1,3):
  
  
  t2, p2 = mannwhitneyu(rater1[str(i)],rater1['3'])
  print "Skin " + str(i) + " vs. Skin 3 p-value: %1.2f " % p2


print "" 

for i in range(1,2):
  
  
  t2, p2 = mannwhitneyu(rater1[str(i)],rater1['2'])
  print "Skin " + str(i) + " vs. Skin 2 p-value: %1.2f " % p2


print ""

T-test for comparing the means of two distributions. This test gave similar results compared to the Mann-Whitney U Test, but is invalid due to the distribution being non normal.

In [ ]:
from scipy import stats
print "Rater 1"
for i in range(1,5):
  
  
  t2, p2 = stats.ttest_ind(rater1[str(i)],rater1['5'],equal_var = False)
  print "Skin " + str(i) + " vs. Skin 5 p-value: %1.2f " % p2 
  
print ""

for i in range(1,4):
  
  
  t2, p2 = stats.ttest_ind(rater1[str(i)],rater1['4'],equal_var = False)
  print "Skin " + str(i) + " vs. Skin 4 p-value: %1.2f " % p2


print ""  

for i in range(1,3):
  
  
  t2, p2 = stats.ttest_ind(rater1[str(i)],rater1['3'],equal_var = False)
  print "Skin " + str(i) + " vs. Skin 3 p-value: %1.2f " % p2


print "" 

for i in range(1,2):
  
  
  t2, p2 = stats.ttest_ind(rater1[str(i)],rater1['2'],equal_var = False)
  print "Skin " + str(i) + " vs. Skin 2 p-value: %1.2f " % p2


print ""

Chisquare test to see if the sampling distribution of the test statistic is a chi-squared distribution when the null hypothesis is true. Without other qualification, 'chi-squared test' often is used as short for Pearson's chi-squared test.

among yellow, yellow-red, and red card recipients, the skin tone breakdown is the same as the global dataset of players.

I started by separating tone into just lighter (<0.5) or darker (>=0.5).

In [ ]:
from scipy.stats import chisquare

# expected proportions
props = [ ( len(df[df['tone'] < 0.5]) ) ,
          ( len(df[df['tone'] >= 0.5]) )]

obs_y = df[df['yellowCards'] <= 1]
obs_yell = [ ( len(obs_y[obs_y['tone'] < 0.5]) / len(obs_y) * len(df) ) ,
             ( len(obs_y[obs_y['tone'] >= 0.5]) / len(obs_y) * len(df)  )]
print(props)
print(obs_yell)

# Yellow cards
print(chisquare(
    obs_yell, 
    f_exp=props)) 

obs_y = df[df['yellowReds'] <= 1]
obs_yellred = [ ( len(obs_y[obs_y['tone'] < 0.5]) / len(obs_y) * len(df) ) ,
             ( len(obs_y[obs_y['tone'] >= 0.5]) / len(obs_y) * len(df)  )]
print(props)
print(obs_yellred)

# Yellow reds
print(chisquare(
    obs_yellred, 
    f_exp=props)) 

obs_y = df[df['redCards'] <= 1]
obs_red = [ ( len(obs_y[obs_y['tone'] < 0.5]) / len(obs_y) * len(df) ) ,
             ( len(obs_y[obs_y['tone'] >= 0.5]) / len(obs_y) * len(df)  )]
print(props)
print(obs_red)

# Red cards
print(chisquare(
    obs_red, 
    f_exp=props))

[lighter, darker]
[90302, 25678]
[89788.15188349076, 26191.848116509238]
Power_divergenceResult(statistic=13.206692886609037, pvalue=0.0002789511616150587)
[90302, 25678]
[90296.68529442203, 25683.314705577977]
Power_divergenceResult(statistic=0.0014128074034621863, pvalue=0.9700167096939366)
[90302, 25678]
[90301.1290294762, 25678.87097052381]
Power_divergenceResult(statistic=3.79429816402156e-05, pvalue=0.9950852317623295)
p-values suggest that players with darker skin tone received more yellow cards than we would expect by chance. Yellow-reds and red cards were on par with the skin tone breakdown.

I repeated the analysis with a finer breakdown by tone.

Correlations

In [ ]:
features = df[['club','leagueCountry','height','weight','position','games','victories','ties','defeats','goals','age','tone']]
features.corr()

In [ ]:
# chi2_contingency
from scipy.stats import chi2_contingency
chi2_contingency(c_tab)

from scipy.stats import chi2_contingency

chi2_contingency(c_tab)


Out[50]:

(2.5648518623027297,
 0.1092627921379188,
 1,
 array([[600.39293286,  51.60706714],
        [702.60706714,  60.39293286]]))


These results suggest that the p-value of 0.10 is not significant, and we would not reject the null hypothesis that whether or not a player received any redcards is independent of their skin-tone being light or dark. The p-value of 0.10 indicates there is an approximately 10% chance we would see this result due to random chance and not because of a specific trend in this data.


The array from the chi-square output is the expected values if getting a red-card was independent of race. For the dark skin-toned column (column 2), this means we would expect 51 people to NOT get red-cards and 60 people to get redcars. In our observed data, we got 43 people who did NOT get red-cards, and 69 peopple who did. Thus, our observed data is not that far off from the idealized case, and we are not confident saying that our observed data deviates from the expected scenario when race does not matter.

T-Test
Run a t-test testing the difference between the 2 groups. This test assumes the number of redcards is approximately normally distributed for both groups- check to make sure dist is normal for both groups you are comparing:

In [ ]:
# Simple chart to look for normal distribution- do one for each population
plt.hist(df_player['total_reds'][df_player.dark==0])
plt.xlabel("number of red cards")
plt.ylabel("Frequency for light players")

# If not normally distributed:
You can try a log-transformation UNLESS you have 0 values, in which case a log-transformaiton will not work. In that case (and if log-transformation still produces non-normal distributions) a non-parametric test must be used.

A non-parametric test that is similar to the t-test is the Mann-Whitney rank-sum test.

The Mann-Whitney test is also fine to use on imbalanced data.

In [ ]:
from scipy.stats import mannwhitneyu

light=df_player['total_reds'][df_player.dark==0]
dark=df_player['total_reds'][df_player.dark==1]
stat, p = mannwhitneyu(light, dark)
print('Statistics=%.3f, p=%.3f' % (stat, p))

In [ ]:
Statistics=68377.500, p=0.119
Again, the p-value of 0.12 indicates there is no reason to reject the null hypothesis that these two groups (light and dark) have the same distriibution of number of red-cards received by a player.

Thus, I would say that "skin tone does not affect the total number of red-cards a player receives".